In [ ]:
#Defining the classification model
lgb = LGBMClassifier()
lgb.fit(X_train_res, y_train_res)

In [ ]:
#Finding the predicted probability of the target variable
Y_predict_proba=lgb.predict_proba(X_test)[:,1]

In [ ]:
def threshold_evaluation(Y_prob,threshold=0):
 
    '''
    
    Y_prob : the predicted probablity value of the target variable
    
    threshold : the cut-off probablity value for classifying the predicted value as 0 or 1.By default the threshold 
    takes the list.
    
    return : returns the Dataframe of evaluation metrices as the output
    
    '''
    threshold_list=np.linspace(start = 0, stop = 1, num = 11)
    
    if threshold!=0:
        threshold_list=[]
        threshold_list.append(threshold)
    
    result=[]
    
     # converting the predicted value into binary class based on a threshold value
        
    for j in threshold_list:
        
        Y_predict=[]
         
        for i in Y_prob:
            
            if i >= j:
                    
                Y_predict.append(1)
                
            else:
                    
                Y_predict.append(0)
        
        
        y_predict=pd.Series(Y_predict)

    # Calculating the evaluation metrics

        tn, fp, fn, tp = confusion_matrix(y_test,y_predict).ravel()

        
        recall = tp / (tp + fn)
        
        if tp == fp == 0:
        
            precision = 0
        
            true_positive_rate = 0
        
        else:
        
            precision = tp / (tp + fp)
            
            true_positive_rate = tp / (tp + fn)
            
        if precision == recall == 0:
        
            f1 = 0
        
        else:
        
            f1 = 2 * (precision * recall) / (precision + recall)
        
        false_positive_rate = fp / (fp + tn)

        specificity = tn / (tn + fp)

        result.append([j,tn, fp, fn, tp,recall,precision,f1,true_positive_rate,false_positive_rate,specificity])
    
    result=pd.DataFrame(result)
    
    result.columns=["threshold","tn", "fp", "fn", "tp","recall", "precision", "f1", "true_positive_rate","false_positive_rate","specificity"]

    return(result)

    #return [threshold,tn, fp, fn, tp,recall,precision,f1,true_positive_rate,false_positive_rate,specificity]

In [ ]:
roc=threshold_evaluation(Y_predict_proba)
roc

In [ ]:
# Plotting the ROC Curve

figsize(10, 8)
plt.style.use('seaborn-dark-palette')
thresholds = [round(t,2) for t in roc['threshold']]
plt.plot(roc['false_positive_rate'], roc['true_positive_rate'], 'bo-', label = 'model');
plt.plot(list(np.linspace(0, 1, num = 10)), list(np.linspace(0, 1, num = 10)), 'ro--', label = 'naive classifier');
for x, y, s in zip(roc['false_positive_rate'], roc['true_positive_rate'], thresholds):
    plt.text(x - 0.04,y + 0.02, s, fontdict={'size': 14});

plt.legend(prop={'size':14})
plt.ylabel('True Positive Rate', size = 16); plt.xlabel('False Positive Rate', size = 16);
plt.title('Receiver Operating Characteristic Curve', size = 20);

# Finding Area under the ROC Curve

print('ROC Score : '+str(roc_auc_score(y_test, lgb.predict_proba(X_test)[0:,1]))) 